# What do we want to know?

# Are we getting bad execution here?

In [27]:
import pandas as pd
import plotly.express as px

from mainnet_launch.constants import *
from mainnet_launch.database.schema.full import IncentiveTokenPrices, IncentiveTokenSwapped
from mainnet_launch.database.schema.postgres_operations import get_full_table_as_df
from mainnet_launch.database.schema.views import get_incentive_token_sold_details

df = get_incentive_token_sold_details()

In [60]:
CHAIN_BASE_ASSET_GROUPS[(ETH_CHAIN, WETH)]

(AutopoolConstants(name='autoETH', symbol='autoETH', autopool_eth_addr='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', autopool_eth_strategy_addr='0xf5f6addB08c5e6091e5FdEc7326B21bEEd942235', solver_rebalance_plans_bucket='v2-gen3-eth-mainnet-solve-v2gen3ethmainnetsolverau-7ju0ra8aoznh', chain=ChainData(name='eth', block_autopool_first_deployed=20722908, chain_id=1, start_unix_timestamp=1726365887, tokemak_subgraph_url='https://subgraph.satsuma-prod.com/108d48ba91e3/tokemak/v2-gen3-eth-mainnet/api'), base_asset='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', block_deployed=20722908, data_from_rebalance_plan=False, base_asset_symbol='ETH', start_display_date='9-14-2024', base_asset_decimals=18),
 AutopoolConstants(name='autoLRT', symbol='autoLRT', autopool_eth_addr='0xE800e3760FC20aA98c5df6A9816147f190455AF3', autopool_eth_strategy_addr='0x72a726c10220280049687E58B7b05fb03d579109', solver_rebalance_plans_bucket='v2-gen3-eth-mainnet-solve-v2gen3ethmainnetsolverau-cb9s4hq4f8sv', chain=ChainDa

In [47]:
base_df = df[df["chain_id"] == ETH_CHAIN.chain_id].copy()

base_df["label"] = base_df.apply(lambda row: f"{row['sell']} -> {row['buy']}", axis=1)
base_weth_sales = [a for a in base_df["label"].value_counts().index if " -> USDC" in a]
base_weth_sales_df = base_df[base_df["label"].isin(base_weth_sales)].copy()
base_weth_sales_df["price_diff_pct"] = (
    (base_weth_sales_df["actual_execution"] - base_weth_sales_df["third_party_price"])
    / base_weth_sales_df["third_party_price"]
    * 100
)
px.box(
    base_weth_sales_df, x="label", y="price_diff_pct", points="all", title="Incentive Token Sales vs 3rd Party Price"
)

In [56]:
import plotly.express as px

fig = px.ecdf(
    base_weth_sales_df,
    x="price_diff_pct",
    color="label",
    title="ECDF of Incentive Token Sales vs 3rd Party Price",
    markers=True,  # optional, to show dots
)

# Optional: add vertical reference line at 0% diff
fig.add_vline(x=0, line_width=1, line_dash="dash", line_color="red")

fig.show()

In [54]:
import plotly.express as px

fig = px.histogram(
    base_weth_sales_df,
    x="price_diff_pct",
    facet_col="label",  # split into subplots per label
    facet_col_wrap=3,  # wrap into rows after 3 columns
    nbins=50,
    title="Incentive Token Sales vs 3rd Party Price by Label",
    color="label",  # optional: color by label
    opacity=0.75,
)

# Optional: add 0% reference line to each subplot
fig.update_yaxes(matches=None)  # allow independent y-scales
fig.add_vline(x=0, line_width=1, line_dash="dash", line_color="red")

fig.show()